# Testing eSCAPE installation

<div class="alert alert-block alert-info">
This example is for testing purpose and should be used to check that your installation is producing expected results.
</div>

Once all the [installation process](https://escape-model.github.io/2018/09/installation/) has been successfull, you could use this notebook to test and compare your output and runtime with expected ones.


## Content
   - [Expected output and runtime values](#Expected-output-and-runtime-values)
   - [Jupyter environment](#Jupyter-environment)
   - [MPI environment](#MPI-environment)
   - [Paraview visualisation](#Paraview-visualisation)


# Expected output and runtime values

The test is first done within the _Jupyter_ environment. 

It consists in a surface made of about 15000 points (`data/elev.vtu`). A constant sealevel at 500 m elevation is defined as well as a uniform precipitation (1 m/yr). The input file (`input_test.yml`) has the following parameters:

```yml

name: simple model for testing installation

domain:
    filename: ['data/elev.vtu','Z']
    flowdir: 4

time:
    start: 0.
    end: 100.
    tout: 100.
    dt: 100.

sea:
    position: 500.

climate:
    - start: 0.
      uniform: 1.0

spl:
    Ke: 1.e-5

diffusion:
    hillslopeK: 5.e-2
    streamK: 500.
    oceanK: 200.
    maxIT: 50

output:
    dir: 'testinstall'
    makedir: False
    
```

Definition of these parameters is provided in the [input file](https://escape-model.github.io/2018/09/input-files/) page.

The model is only ran for one time step (100 years) and the following values at the end of the run are evaluated:
+ `EDmax`: maximum deposition thickness in metres
+ `EDmin`: minimum deposition thickness in metres
+ `hmax`: maximum elevation in metres
+ `hmin`: minimum elevation in metres

The values could potentially be _slightly different between different compilers_ but will be quite close if your installation is successfull.

### Model output values for comparison

In [ ]:
EDmaxInstall = 8.4144388862
EDminInstall = -1.08636116119
hmaxInstall = 1264.82966136
hminInstall = 19.2141465268

### Runtime values for comparison

One additional test consists in estimating the simulation runtime. Here again values will be different depending on the installed libraries and the computer/HPC specs.

Nevertheless the following runtimes give an idea of the expected duration for serial and parallel runs:

+ **serial runtime**: around 8 seconds
+ **parallel runtime**: around 2.5 seconds with 4 CPUs

# Jupyter environment

We first test the serial version of **eSCAPE**. This is done by importing **eSCAPE** and defining the super class `LandscapeEvolutionModel`.

This class is the main entry point to **eSCAPE** model and takes 3 parameters as shown in the docstring...

```python
Instantiates eSCAPE model object and performs surface processes evolution.

This object contains methods for the following operations:
 - initialisation of eSCAPE mesh based on input file options.
 - computation of surface processes
 - cleaning/destruction of PETSC objects

Args
    filename : YAML input file
    verbose : True/False
        Output option for model main functions
    showlog : True/False
        Output option for PETSC logging file

Returns:
    LandscapeEvolutionModel : object
```

<div class="alert alert-block alert-info">
Here we will set the verbose parameter to True</div>

In [ ]:
import eSCAPE as sim
model = sim.LandscapeEvolutionModel('input_test.yml',True,False)

This first cell should output the time of each major calls, and should be similar to what follows:
    
```docstring
The following model will be run:     simple model for testing installation
Reading mesh information (0.00 seconds)
Create DMPlex (0.04 seconds)
Distribute DMPlex (0.01 seconds)
Distribute field to DMPlex (0.00 seconds)
Defining Petsc DMPlex (0.11 seconds)
Voronoi creation (0.04 seconds)
Tesselation (0.02 seconds)
Finite volume mesh declaration (0.17 seconds)
Priority-flood algorithm initialisation (0.01 seconds)
Update External Forces (0.00 seconds)
--- Initialisation Phase (1.84 seconds)
```

We then run the surface processes using the following function `runProcesses`

```python
Run eSCAPE Earth surface processes.

This function contains methods for the following operations:
 - calculating flow accumulation
 - erosion/deposition induced by stream power law
 - depression identification and pit filling
 - stream induced deposition diffusion
 - hillslope diffusion
```

In [ ]:
model.runProcesses()

The cell outputs should have similar runtimes for each function as the ones provided below:

```docstring
Flow Direction declaration (0.00 seconds)
Compute Flow Accumulation (0.74 seconds)
Creating outputfile (0.03 seconds)
+++ Output Simulation Time: 0.00 years
Compute Stream Power Law (1.51 seconds)
Pit filling algorithm (0.01 seconds)
Pit parameters definition (0.00 seconds)
Fill Pit Depression (0.00 seconds)
Compute Sediment Diffusion (0.32 seconds)
Compute Hillslope Processes (0.01 seconds)
Update External Forces (0.00 seconds)
--- Computational Step (2.63 seconds)
Flow Direction declaration (0.00 seconds)
Compute Flow Accumulation (0.74 seconds)
Compute Stream Power Law (1.47 seconds)
Pit filling algorithm (0.01 seconds)
Pit parameters definition (0.00 seconds)
Fill Pit Depression (0.00 seconds)
Compute Sediment Diffusion (0.30 seconds)
Compute Hillslope Processes (0.01 seconds)
Update External Forces (0.00 seconds)
Creating outputfile (0.02 seconds)
+++ Output Simulation Time: 100.00 years
--- Computational Step (2.55 seconds)
```

We will now extract the output variables that will be compared with the expected ones provided at the top of this notebook:

In [ ]:
EDmax = model.cumED.max()[1]
EDmin = model.cumED.min()[1]
hmax = model.hGlobal.max()[1]
hmin = model.hGlobal.min()[1]

The cell below will plot the comparisons between expected and computed values for
+ `EDmax`: maximum deposition thickness in metres
+ `EDmin`: minimum deposition thickness in metres
+ `hmax`: maximum elevation in metres
+ `hmin`: minimum elevation in metres

In [ ]:
print('-------------------------------------------------------------')
print('                 Testing eSCAPE installation                 ')
print('-------------------------------------------------------------')
print
str_fmt = "{:35} {:9}"
print(str_fmt.format('Maximum deposition thickness (m):', EDmax))
print(str_fmt.format('Minimum deposition thickness (m):', EDmin))
print
print(str_fmt.format('Maximum elevation (m):', hmax))
print(str_fmt.format('Minimum elevation (m):', hmin))
print
print('-------------------------------------------------------------')
print('           Comparison with expected installation             ')
print('-------------------------------------------------------------')
print
str_fmt = "{:35} {:5.2f}"
print(str_fmt.format('Difference in max deposition thickness (m):', abs(EDmax-EDmaxInstall)))
print(str_fmt.format('Difference in min deposition thickness (m):', abs(EDmin-EDminInstall)))
print
print(str_fmt.format('Difference in maximum elevation (m):', abs(hmax-hmaxInstall)))
print(str_fmt.format('Difference in minimum elevation (m):', abs(hmin-hminInstall)))
print
print
print('-------------------------------------------------------------')
print('                      ending eSCAPE test                     ')
print('-------------------------------------------------------------')

We expect the results of the comparisons to be close to 0.0! 

We now run the last **eSCAPE** function `destroy()`:
```python
Destroy PETSc DMPlex objects and associated Petsc local/global Vectors and Matrices.
Safely quit eSCAPE model.
```

In [ ]:
model.destroy()

The total run time should be close to 8 seconds...

# MPI environment

We perform a similar test for the MPI installation this time using the provided `run_escape.py` python file:

```python
import argparse
import eSCAPE as sim
from petsc4py import PETSc
MPIrank = PETSc.COMM_WORLD.Get_rank()

# Parsing command line arguments
parser = argparse.ArgumentParser(description='This is a simple entry to run eSCAPE model.',add_help=True)
parser.add_argument('-i','--input', help='Input file name (YAML file)',required=True)
parser.add_argument('-v','--verbose',help='True/false option for verbose', required=False,action="store_true",default=False)
parser.add_argument('-l','--log',help='True/false option for PETSC log', required=False,action="store_true",default=False)

args = parser.parse_args()
if args.verbose:
  print("Input file: {}".format(args.input))
  print(" Verbose is on? {}".format(args.verbose))
  print(" PETSC log is on? {}".format(args.log))

# Reading input file
model = sim.LandscapeEvolutionModel(args.input,args.verbose,args.log)

# Running model
model.runProcesses()

# Test values
EDmax = model.cumED.max()[1]
EDmin = model.cumED.min()[1]
hmax = model.hGlobal.max()[1]
hmin = model.hGlobal.min()[1]

if MPIrank == 0:
    EDmaxInstall = 8.4144388862
    EDminInstall = -1.08636116119
    hmaxInstall = 1264.82966136
    hminInstall = 19.2141465268
    print
    print('-------------------------------------------------------------')
    print('                 Testing eSCAPE installation                 ')
    print('-------------------------------------------------------------')
    print
    str_fmt = "{:35} {:9}"
    print(str_fmt.format('Maximum deposition thickness (m):', EDmax))
    print(str_fmt.format('Minimum deposition thickness (m):', EDmin))
    print
    print(str_fmt.format('Maximum elevation (m):', hmax))
    print(str_fmt.format('Minimum elevation (m):', hmin))
    print
    print('-------------------------------------------------------------')
    print('           Comparison with expected installation             ')
    print('-------------------------------------------------------------')
    print
    str_fmt = "{:35} {:5.2f}"
    print(str_fmt.format('Difference in max deposition thickness (m):', abs(EDmax-EDmaxInstall)))
    print(str_fmt.format('Difference in min deposition thickness (m):', abs(EDmin-EDminInstall)))
    print
    print(str_fmt.format('Difference in maximum elevation (m):', abs(hmax-hmaxInstall)))
    print(str_fmt.format('Difference in minimum elevation (m):', abs(hmin-hminInstall)))
    print
    print
    print('-------------------------------------------------------------')
    print('                      ending eSCAPE test                     ')
    print('-------------------------------------------------------------')

# Cleaning model
model.destroy()
```

The file requires the input file (`input_test.yml`) and we also set the verbose and log functionalities on.

To call **eSCAPE** in parallel the following command needs to be used:

In [ ]:
!mpirun -np 4 python run_escape.py -i input_test.yml -v -l

The model output provided above contains the comparisons with the expected values for elevation and erosion deposition as well as the runtime (this later should be around 2.5 seconds).

For comparisons between obtained log from your installation and expected log a file called `test.out` is provided in the `test` directory and could be used to check any potential issues.

If you are unsure about the result of your test, <a href="MAILTO:tristan.salles@sydney.edu.au?subject=eSCAPE User Group&body=Please send me an invite to join the eSCAPE User Group">Join eSCAPE User Group</a>

# Paraview visualisation

The model outputs are located in the `testinstall` folder and consist of a time series file named `eSCAPE.xdmf` and 2 folders (`h5` and `xmf`). The XDMF file is the main entry point for visualising the output and should be sufficient for most users. 

The file can be opened with the **Paraview** software. 

<div class="alert alert-block alert-info">
The video below provides an example of how to visualise the output of this simulation.</div>

In [ ]:
from IPython.display import HTML
HTML('<video controls="controls" width="800" height="600" name="eSCAPE Paraview" src="data/paraview_test.mov"></video>')

After loading the file in paraview, we perform 2 operations:
+ first we use the `wrap by scalar` filter to create a 2D representation of the surface
+ then we define a `contour` line corresponding to the sea-level position (500 m as set in the `YAML` input file)